# **ATIVIDADE Projeto 3**
### M12- Fundamentos da Descoberta de Dados

Nesse projeto trabalharemos com a base de dados de produtos de um supermercado do Chile. A ideia é que vocês apliquem os conceitos estatísticos vistos no último módulo, mais os conceitos de visualizações de dados através de gráficos e finalizem publicando no seu github!
Os campos do nosso dataframe são:

**Title:** Nome do produto. \
**Marca:** A marca do produto.\
**Preco_Normal:** O preço em que o produto costuma ser vendido quando não há desconto.\
**Preco_Desconto:** O preço vendido após o desconto ser aplicado.\
**Preco_Anterior:** Preço em que era comercializado o produto antes do desconto aplicado.\
**Desconto:** Total de desconto aplicado.

As colunas que aparecem com valores 0 são para os produtos onde não tivemos descontos aplicados.

### **Objetivos**
#### 1 - Encontre a média e mediana dos preços por categoria de produto.
 Identifique as categorias que parecem ter um valor de média abaixo ou acima da mediana. 
#### 2 - Calcule o desvio padrão por categoria de produto. 
Qual o comportamento da média e mediana nas categorias com maior desvio? 

#### 3 - Plot um boxplot da distribuição do Preco_Normal para a categoria que você identificou que tem o maior desvio padrão. Como é a distribuição desses dados segundo o boxplot? Você identifica muitos outliers?

#### 4 - Plote um gráfico de barras onde temos a média de descontos por categoria.

#### 5 - Plote um gráfico de mapa interativo agrupando os dados por categoria, marca e trazendo a média de desconto.

In [88]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
df = pd.read_csv("MODULO7_PROJETOFINAL_BASE_SUPERMERCADO.csv", delimiter=',')


### Preparação e organização dos dados
A base de dados original é o descritivo de um supermercado, apresentando 1107 linhas relativa aos produtos e 7 colunas para categorias.
**Categorias preços*
Os dados apresentam três categorias de preços: Normal, Com Desconto e Anterior. A contagem dos produtos com esses preços está exibida na **tabela 1**.

| Produtos  | Total  |
|-----------|-----------|
| Sem preço   | 50   |
| Preço antigo e com desconto   | 118   |
| preço normal   | 813  |
| preço desconto   | 126   |
**tabela 1**: relação preco e quantidade

Assim, para uma melhro compreenção nas analises seguintes realizamos a seguintes operações:
1. Removemos os produtos sem preço
2. Criamos duas novas colunas: **Preco_padrão** e **Preco_Promo**. Seus valores seguem a lógica apresentada na **tabela 2**.

| Preco_normal  | Preço_desconto  | Preco_anterior  | Preço_padrão | Preco_promo  |
|-----------|-----------|-----------|-----------|-----------|
| 0   | A   | B   | B   | A  |
| A   | 0   | 0   | A   | A   |
| 0  | A   | 0  | A   | A   |
**tabela 2**: Lógica de organização das novas colunas.

In [89]:
#dimensão 1107x7
#valores unicos 1065
n_cat=df['Categoria'].nunique() # 7 categorias
L_cat=df['Categoria'].unique()  # 
print(df.dtypes)
#df.head(25)
#df.info()

title             object
Marca             object
Preco_Normal       int64
Preco_Desconto     int64
Preco_Anterior     int64
Desconto           int64
Categoria         object
dtype: object


In [90]:
S_da=S_n=S_d=S_a=S_o=0
'''
unicas possibilidade de preço encontradas nos dados.
cond1: Desconto + preço antigo
cond2: preço normal  
cond3: Desconto

'''
for i in df.index:
    cond1 = (df.loc[i, 'Preco_Normal'] == 0) & (df.loc[i, 'Preco_Desconto'] != 0) & (df.loc[i, 'Preco_Anterior'] != 0)
    cond2 = (df.loc[i, 'Preco_Normal'] != 0) & (df.loc[i, 'Preco_Desconto'] == 0) & (df.loc[i, 'Preco_Anterior'] == 0)
    cond3 = (df.loc[i,'Preco_Normal'] == 0 )&  (df.loc[i,'Preco_Desconto']!=0)&  (df.loc[i,'Preco_Anterior']==0)  

    if(df.loc[i,'Preco_Normal'] != 0 ) and (df.loc[i,'Preco_Desconto']!=0)and (df.loc[i,'Preco_Anterior']!=0):
        print(i,'3 Precos')
    elif(df.loc[i,'Preco_Normal'] != 0 )and  (df.loc[i,'Preco_Desconto']!=0)and  (df.loc[i,'Preco_Anterior']==0):
        print(i,'Normal e Desconto')
    if cond1: 
        S_da+=1
    if cond2:
        S_n+=1
    if cond3 :
        S_d+=1
    elif(df.loc[i,'Preco_Normal'] == 0 )and  (df.loc[i,'Preco_Desconto']==0)and  (df.loc[i,'Preco_Anterior']!=0):
        S_a+=1
    elif(df.loc[i,'Preco_Normal'] == 0 )and  (df.loc[i,'Preco_Desconto']==0)and  (df.loc[i,'Preco_Anterior']==0):
       S_o+=1
        
print('Desconto & anterior :',S_da) 
print('Só preço Normal     :',S_n)
print('Só com Desconto     :',S_d)
print('Sem Preço    :',S_o)
print('Só anterior   :',S_a)

#Remover linhas onde não há preço
df.drop(df[(df['Preco_Normal'] == 0) & (df['Preco_Desconto'] == 0) & (df['Preco_Anterior'] == 0)].index, inplace=True)

Desconto & anterior : 118
Só preço Normal     : 813
Só com Desconto     : 126
Sem Preço    : 50
Só anterior   : 0


In [91]:
#criar categoria preco vendido
#preço normal Ou preco com desconto serão incluidas
df['Preco_Padrao'] = 0.0
df['Preco_Promo'] = 0.0

for i in df.index:
    cond1 = (df.loc[i, 'Preco_Normal'] == 0) & (df.loc[i, 'Preco_Desconto'] != 0) & (df.loc[i, 'Preco_Anterior'] != 0)
    cond2 = (df.loc[i, 'Preco_Normal'] != 0) & (df.loc[i, 'Preco_Desconto'] == 0) & (df.loc[i, 'Preco_Anterior'] == 0)
    cond3 = (df.loc[i,'Preco_Normal'] == 0 )&  (df.loc[i,'Preco_Desconto']!=0)&  (df.loc[i,'Preco_Anterior']==0)
    if cond1:
         df.loc[i, 'Preco_Padrao'] = df.loc[i, 'Preco_Anterior']
         df.loc[i, 'Preco_Promo'] = df.loc[i, 'Preco_Desconto']  
    if cond2:
         df.loc[i, 'Preco_Padrao'] = df.loc[i, 'Preco_Normal']
         df.loc[i, 'Preco_Promo'] = df.loc[i, 'Preco_Normal']  
    if cond3:
         df.loc[i, 'Preco_Padrao'] = df.loc[i, 'Preco_Desconto']
         df.loc[i, 'Preco_Promo'] = df.loc[i, 'Preco_Desconto']  


### **Média e mediana por categoria**
#### 1) **Quantas categorias existem ? Como é o comportamento da média e mediana das categorias?**\
   Os dados apresentam um total de **7 categorias**. São elas: **lacteos, belleza-y-cuidado-personal,congelados,frutas, comidas-preparadas,verduras, instantaneos-y-sopas**. Para avaliar os valores de média e mediana, agrupamos os preços em duas categorias devido à presença de descontos. Isso permite uma análise mais precisa do preço de venda dos produtos naquele momento.As duas novas categorias criadas são:\
**Preço_Padrão**: composta pelos valores das colunas **Preço_Normal + Preço_Anterior**.\
**Preço_Promo**: composta pelos valores das colunas **Preço_Normal + Preço_Desconto**.\
Essa reorganização foi realizada para evitar a inclusão de valores zero no cálculo das médias e medianas, o que poderia distorcer os resultados. Atribuir um valor zero implicaria que alguns produtos seriam gratuitos, o que não reflete a realidade. Assim, média e mediana tem comportamentos distintos conforme apresentados na **Tabela 3**.

| Categoria                  |   media_padrao |   mediana_padrao |   diferenca_padrao |   STD_padrao |   media_promo |   mediana_promo |   diferenca_promo |   STD_promo |
|:---------------------------|---------------:|-----------------:|-------------------:|-------------:|--------------:|----------------:|------------------:|------------:|
| belleza-y-cuidado-personal |       2957.97  |             2690 |          0.0905915 |      2119.66 |      2834.88  |            2543 |         0.102961  |     2118    |
| comidas-preparadas         |       3823.96  |             3890 |          0.0169778 |      1631.47 |      3780.48  |            3590 |         0.0503847 |     1635.89 |
| congelados                 |       3150.28  |             3059 |          0.0289755 |      1836.1  |      2996.25  |            3059 |         0.0205129 |     1781.78 |
| frutas                     |       1724.47  |             1195 |          0.307035  |      1639.15 |      1724.47  |            1195 |         0.307035  |     1639.15 |
| instantaneos-y-sopas       |        883.105 |              459 |          0.480243  |      1106.01 |       883.105 |             459 |         0.480243  |     1106.01 |
| lacteos                    |       2581.43  |             1279 |          0.504538  |      3932.68 |      2564.02  |            1279 |         0.501173  |     3898.28 |
| verduras                   |       1343.3   |             1180 |          0.121564  |      1012.7  |      1343.3   |            1180 |         0.121564  |     1012.7  |

**Tabela 3**: Média, mediana, desvio padrão(STD) e diferença relativa(diff) para cada categoria com preço padrão e promocional.\

Para avaliar a média e mediana, usamos a diferença relativa entre os valores obtidos. Ele é calculado pela seguinte expressão
$$d_{rel}=\frac{abs(\nu-md)}{c}$$
onde $\nu$ é a média, $md$ é a mediana,$ c$ é o maior valor entre média e mediana. Essa informação nos diz percentualmente quão  distante está os valores, partindo do maior valor encontrado. Assim, **adotaremos o critério  $d_{rel} <10%$, mediana e média são aproximadamente iguais** e assim desta forma a distribuição é aproximadamente simétrica. Métodos mais elaborados, como o shapiro-wilk, fornecem mais qualidade nesta avaliação, mas para este projeto, não era o objetivo. Assim a avaliação para cada categoria fica:

| Categoria | Preco padrão | Preco Promo | $\mu$ & $md$ |
|----------|----------|----------|----------|
| belleza-y-cuidado-personal   | $$\mu \approx md$$  | $$\mu \neq md$$  | $\mu$ > $md$| 
| comidas-preparadas   | $$\mu \approx md$$     |$$\mu \approx md$$   |$\mu$ < $md$ | 
|congelados   | $$\mu \approx md$$  |$$\mu \approx md$$   |$\mu$ > $md$ | 
| frutas  | $$\mu \neq md$$  | $$\mu \neq md$$   |$\mu$ > $md$ | 
| instantaneos-y-sopas: | $$\mu \neq md$$ | $$\mu \neq md$$   |$\mu$ > $md$ | 
|  lacteos  |$$\mu \neq md$$| $$\mu \neq md$$   |$\mu$ > $md$ | 
| Verduras:  | $$\mu \neq md$$  | $$\mu \neq md$$  |$\mu$ > $md$| 

**Comentário**\
Todas as categorias apresentam valores distintos de média e mediana. Ao avaliar cada categoria percebemos apenas que as categorias **comida-preparada** e **congelados** tem valores muito proximo,o qual podemos afirmar que tem uma distribuição de preço simétrica ou quase normal. As categorias **belleza-y-cuidado-personal** e **verdura** quando avaliamos a diferença relativa entre média e mediana, encontra-se no limite do critério definido para dados simétricos. As demais categorias, pela avaliação feita, não configuram-se como uma distribuição simétrica ou normal. 



In [92]:
##resumo preco padrão/categoria
#média,mediana,std
md_cat_p =df.groupby('Categoria')['Preco_Padrao'].mean()
mdn_cat_p=df.groupby('Categoria')['Preco_Padrao'].median()
std_cat_p=df.groupby('Categoria')['Preco_Padrao'].std()

resumo_padrao = pd.concat([md_cat_p, mdn_cat_p,std_cat_p], axis=1, keys=['media', 'mediana','desvio'])

#dif=abs(md_cat_p-mdn_cat_p)
dif = []
resumo_padrao['diferenca']=0.0
# Loop sobre os 7 elementos
for i in resumo_padrao.index:
    a=float(resumo_padrao.loc[i,'media'] )
    b=float(resumo_padrao.loc[i,'mediana'])
    if a > b:
        resumo_padrao.loc[i,'diferenca']=abs(a - b) / a
        #dif.append(abs(a - b) / a)
    else:
        resumo_padrao.loc[i,'diferenca']=abs(b - a) / b       
#print(resumo_padrao)

In [93]:
#resumo preco promo/categoria
#média,mediana,std

md_cat_p =df.groupby('Categoria')['Preco_Promo'].mean()
mdn_cat_p=df.groupby('Categoria')['Preco_Promo'].median()
std_cat_p=df.groupby('Categoria')['Preco_Promo'].std()

resumo_promo = pd.concat([md_cat_p, mdn_cat_p,std_cat_p], axis=1, keys=['media', 'mediana','desvio'])

#dif=abs(md_cat_p-mdn_cat_p)
dif = []
resumo_promo['diferenca']=0.0
# Loop sobre os 7 elementos
for i in resumo_promo.index:
    a=float(resumo_promo.loc[i,'media'] )
    b=float(resumo_promo.loc[i,'mediana'])
    if a > b:
        resumo_promo.loc[i,'diferenca']=abs(a - b) / a
    else:
        resumo_promo.loc[i,'diferenca']=abs(b - a) / b      
#print(resumo_promo)

In [20]:
#resumo_precos_geral
resumo_precos = pd.concat([resumo_padrao['media'],resumo_padrao['mediana'],resumo_padrao['diferenca'],resumo_padrao['desvio'],
                           resumo_promo['media'],resumo_promo['mediana'],resumo_promo['diferenca'],resumo_promo['desvio']],
                           axis=1, keys=['media_padrao', 'mediana_padrao',
                                        'diferenca_padrao', 'STD_padrao',
                                          'media_promo','mediana_promo',
                                          'diferenca_promo','STD_promo'])
resumo_precos
#markdown_table = resumo_precos.to_markdown()
#print(markdown_table)

,media_padrao,mediana_padrao,diferenca_padrao,STD_padrao,media_promo,mediana_promo,diferenca_promo,STD_promo
Categoria,,,,,,,,
belleza-y-cuidado-personal,2957.966527,2690.0,0.090591,2119.660645,2971.653509,2599.0,0.125403,2072.458634
comidas-preparadas,3823.956522,3890.0,0.016978,1631.471918,4044.232558,3690.0,0.087590,1332.362721
congelados,3150.280851,3059.0,0.028975,1836.104302,3061.386957,3093.0,0.010221,1744.648525
frutas,1724.473684,1195.0,0.307035,1639.151114,1724.473684,1195.0,0.307035,1639.151114
instantaneos-y-sopas,883.105263,459.0,0.480243,1106.007164,949.754717,579.0,0.390369,1119.298972
lacteos,2581.427293,1279.0,0.504538,3932.682504,2696.741176,1449.0,0.462685,3953.012108
verduras,1343.296875,1180.0,0.121564,1012.699625,1457.135593,1250.0,0.142153,972.230682


### **Desvio Padrão**
#### 2) **Calcule o desvio padrão por categoria de produto. Qual o comportamento da média e mediana nas categorias com maior desvio?**
Os valores já foram calculados e estão organizados na  tabela 3. Observamos que as categorias que apresentaram maios desvio padrão foram **Lácteos(3932.68)**,**Belleza y cuidado personal(2119.66)** e **Frutas(1639.15)**. Isso dá indicativos de alta dispersão no valor dos preços, se afastando da média e mediana.
#### Comportamento da Média x Mediana
**Lácteos** : Média >> Mediana (2581.43 x 1279). Indica uma distribuição assimétrica com valores altos puxando a média para cima. 
**Belleza y Cuidado Persona** :Média > Mediana (2957.97 x 2690).Também sugere assimetria para valores mais altos.
**Frutas** Média > Mediana (1724.47 x1195):Novamente assimétrica com valores altos dispersos.

#### Conclusão
Categorias com alto desvio padrão apresentam diferenças significativas entre a média e a mediana, indicando a presença de valores extremos. A média tende a ser maior que a mediana, o que sugere uma assimetria para a direita (valores altos dispersos).No caso dos lácteos e frutas, o impacto dos valores extremos é evidente. De forma geral, o comportamento não muda muito entre preços "padrão" e "promo", mas em algumas categorias há leve redução na média. Uma avaliação mais precisa seria considerar o cálculo dos IQR pra eliminar valores de preços de que seriam outlier dentro das categorias. Este calculo foi realizado e encontra-se em anexo, ao final deste notebook.


### **Boxplot**
#### 3)Plot um boxplot da distribuição do Preco_Normal para a categoria que você identificou que tem o maior desvio padrão. Como é a distribuição desses dados segundo o boxplot? Você identifica muitos outliers?

In [129]:
#Calculo dos quartiles
def print_quart(cat):
    temp=df[df["Categoria"] == cat][["title", "Preco_Padrao","Preco_Promo"]]
    Q1 = temp['Preco_Padrao'].quantile(0.25)
    Q3 = temp['Preco_Padrao'].quantile(0.75)
    IQR=Q3-Q1
    C=1.5
    limite_inferior=Q1-C*IQR
    limite_superior=Q3+C*IQR
    print(f"{cat:<30} {limite_inferior:>15.2f} {limite_superior:>17.2f}")

print(f"{'Categoria':<30} {'limite inferior':>15} {'limite superior':>17}")
print("-" * 64)
print_quart('belleza-y-cuidado-personal')
print_quart('comidas-preparadas')
print_quart('congelados')
print_quart('frutas')
print_quart('instantaneos-y-sopas')
print_quart('lacteos')
print_quart('verduras')

Categoria                      limite inferior   limite superior
----------------------------------------------------------------
belleza-y-cuidado-personal             -542.88           5924.12
comidas-preparadas                     1145.75           6863.75
congelados                            -2784.00           8998.00
frutas                                 -322.50           3057.50
instantaneos-y-sopas                   -206.00           1514.00
lacteos                               -2717.50           5902.50
verduras                                -41.25           2568.75


In [132]:
blz = df[df["Categoria"] == "belleza-y-cuidado-personal"][["title", "Preco_Padrao","Preco_Promo"]]
cmd = df[df["Categoria"] == "comidas-preparadas"][["title", "Preco_Padrao","Preco_Promo"]]
cng= df[df["Categoria"] == "congelados"][["title", "Preco_Padrao","Preco_Promo"]]
fru = df[df["Categoria"] == "frutas"][["title", "Preco_Padrao","Preco_Promo"]]
sop = df[df["Categoria"] == "instantaneos-y-sopas"][["title", "Preco_Padrao","Preco_Promo"]]
lac = df[df["Categoria"] == "lacteos"][["title", "Preco_Padrao","Preco_Promo"]]
ver = df[df["Categoria"] == "verduras"][["title", "Preco_Padrao","Preco_Promo"]]
contagem = df['Categoria'].value_counts()
fru.describe()

,Preco_Padrao,Preco_Promo
count,19.000000,19.000000
mean,1724.473684,1724.473684
std,1639.151114,1639.151114
min,645.000000,645.000000
25%,945.000000,945.000000
50%,1195.000000,1195.000000
75%,1790.000000,1790.000000
max,7990.000000,7990.000000
